In [347]:
# class Word defines a word in dictionary; 
class Word(object):
    def __init__(self, WordName=None, WordIndex=None, WordFrequency=None):
        self.WordName = WordName
        self.WordIndex = WordIndex
        self.WordFrequency = WordFrequency


# class Element defines a word in a bug corpus or in a code corpus; 
class Element(object):
    def __init__(self, ElementName=None, ElementIndex=None, ElementWeight=None):
        self.ElementName = ElementName
        self.ElementIndex = ElementIndex
        self.ElementWeight = ElementWeight


# class Corpus defines a bug corpus or a code corpus; 
class Corpus(object):
    def __init__(self, FileName=None):
        self.FileName = FileName
        self.CorpusMembers = []

In [348]:
def generate_OneBugCorpus(FilePath, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    with open(FilePath) as TextFile:
        BugFile = TextFile.read().split(' ')    


    ThisBugCorpus = Corpus(FilePath) # this file delete word that is not in the word list; 

    for i in range(len(BugFile)):
        flag = False
        for j in range(len(WordList)):
            if BugFile[i] == WordList[j].WordName:
                flag = True

                if WordList[j].WordFrequency == 0:
                    ThisBugCorpus.CorpusMembers.append(Element(BugFile[i], 0, 0))

                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break        
       
        
    return ThisBugCorpus

In [349]:
def generate_OneCodeCorpus(ThisCodeFile, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    ThisCodeCorpus = Corpus(ThisCodeFile[0])
    
    for i in range(len(ThisCodeFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisCodeFile[i] == WordList[j].WordName:
                flag = True
                
                if WordList[j].WordFrequency == 0:
                    ThisCodeCorpus.CorpusMembers.append(Element(ThisCodeFile[i], 0, 0))
                    
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break
    
    
    return ThisCodeCorpus

In [350]:
# this process intends to save all file names in an array; 

import os

path = 'C:\\Users\\take_\\OneDrive\\liming\\where\\BugLocator\\tmp\\BugCorpus\\'

BugFiles = []
# r=root, d=directories, f = bug files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            BugFiles.append(os.path.join(r, file))

print(len(BugFiles))

98


In [351]:

## ----------------------------------- read word list from Wordlist.txt --------------------------------------------
with open("WordList.txt") as TextFile:
    WordlistContent = [line.split() for line in TextFile]

WordList = []
    
for i in range(len(WordlistContent)):    
    WordList.append(Word(WordlistContent[i][0], WordlistContent[i][1], 0))
    
# test whether WordList is correct or not;     
#print(WordList[0].WordName)
#print(WordList[1].WordName)
    
    
## ----------------------------------- read all bug files ------------------------------------------------    
#ThisBugCorpus = generate_OneBugCorpus(BugFiles[0], WordList)
#print(ThisBugCorpus.FileName)
#print(ThisBugCorpus.CorpusMembers)

BugCorpuses = []
for i in range(len(BugFiles)):
    ThisBugCorpus = generate_OneBugCorpus(BugFiles[i], WordList)
    BugCorpuses.append(ThisBugCorpus)

# test result of all bug corpuses;     
# print(BugCorpuses[0])
for i in range(1):
    print(BugCorpuses[i].FileName)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
    print()

## ----------------------------------- read all code files ------------------------------------------------
with open("CodeCorpus.txt") as TextFile:
    CodeList = TextFile.read().split()   

#print(len(CodeList))
CodeFiles = []
ThisCodeFile = []

for i in range(len(CodeList)):
    if CodeList[i][-5:] == ".java":        
        CodeFiles.append(ThisCodeFile)
        ThisCodeFile = []
        ThisCodeFile.append(CodeList[i])
    else:
        ThisCodeFile.append(CodeList[i])

CodeFiles.pop(0) # delete the first element;         

#print(len(CodeFiles))
#print(CodeFiles[0])

#ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[0], WordList)
#print(ThisCodeCorpus)

CodeCorpuses = []
for i in range(len(CodeFiles)):
    ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[i], WordList)
    CodeCorpuses.append(ThisCodeCorpus)

# test all code corpuses generated; 
print(CodeCorpuses[0].FileName)
#print(CodeCorpuses[0].CorpusMembers)
for i in range(len(CodeCorpuses[0].CorpusMembers)):
    print(CodeCorpuses[0].CorpusMembers[i].ElementName, end = ',')
print()

C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100040.txt
due,imagelist,build,ui,perform,test,window,run,swt,number,eclips,debug,elaps,process,ms,averag,increas,includ,time,perspect,show,numer,icon,menu,toolbar,os,ilc,mirror,singl,make,
org.eclipse.swt.accessibility.ACC.java
acc,contain,constant,us,defin,access,object,sinc,state,select,multiselect,focus,press,check,expand,collaps,hottrack,busi,readonli,invis,offscreen,sizeabl,link,role,client,area,xa,window,menubar,menu,xb,menuitem,xc,separ,tooltip,xd,scrollbar,dialog,label,pushbutton,checkbutton,radiobutton,combobox,text,toolbar,list,listitem,tabl,tablecel,tablecolumnhead,deprec,tablecolumn,tablerowhead,tree,treeitem,tabfold,tabitem,progressbar,slider,childid,multipl,insert,delet,


In [354]:
import math

with open(BugFiles[0]) as TextFile:
    BugFile = TextFile.read().split(' ')    
    
#print(BugFile)

ThisWord = "build"
# count ftd, the frequency of term t in document d;
ftd = 0
for i in range(len(BugFile)):
    if BugFile[i] == ThisWord:
        ftd = ftd + 1
        
print("ftd:", ftd)

# #docs, total number of documents; 
NumDocs = len(BugFiles) + len(CodeCorpuses);
print("#docs:", NumDocs)

# count nt, the number of docuement that contains t;
# count ntBug, the number of document that contains t in bug files; 
ntBug = 0 
for i in range(len(BugFiles)):
    with open(BugFiles[i]) as TextFile:
        ThisBugFile = TextFile.read().split(' ') 
    
    flag = False    
    
    for j in range(len(ThisBugFile)):
        if ThisWord == ThisBugFile[j]:
            flag = True
            break
    
    if flag == True:
        ntBug = ntBug + 1
    
#print("ntBug:", ntBug)

# count ntCode, the number of document that contains t in code files; 
ntCode = 0
for i in range(len(CodeCorpuses)):
    
    ThisCodeFile = CodeCorpuses[i]
    
    flag = False
    
    for j in range(len(ThisCodeFile.CorpusMembers)):
        if ThisWord == ThisCodeFile.CorpusMembers[j].ElementName:
            flag = True
            break
            
    if flag == True:
            ntCode = ntCode + 1
    
#print("ntCode:", ntCode)

nt = ntBug + ntCode
print("nt:", nt)

w = (math.log(ftd)+1) * math.log(NumDocs/nt)
print(w)

ftd: 1
#docs: 581
nt: 19
3.4203117776854706
